In [1]:
def circular(num,pos):
    '''
    Function for circular shifting of bits.
    
    Parameters:
    num (int): Integer number to be circularly shifted.
    pos (int): No. of bits to be circularly shifted.
    
    Returns:
    num (int): Circularly shifted number.
    '''
    num=num<<pos
    rem=num//(2**32)
    num=num-rem*(2**32)+rem
    return num

In [2]:
def text2blocks(text):
    '''
    Creating the hex equivalent of input text string
    
    Parameters:
    text (string): Input text string.
        
    Returns:
    inp (string): Hexadecimal equivalent of string
    '''
    s=''
    for i in text:
        s+=hex(ord(i))[2:]
    s+='80'
    l=hex(len(text)*8)[2:]
    l=l.zfill(16)
    inp=s+'0'*(128-(len(s)+len(l))%128)+l
    return inp

In [3]:
def initialise_parameters():
    '''
    Initialises the constants and functions of SHA-1.
    
    Parameters:
    None
        
    Returns:
    h and hprev (list of hexadecimal numbers): Initial value of hash
    k (list of hexadecimal number): Constants required for every set of 20 rounds
    func (list of functions): Functions required for every set of 20 rounds
    '''
    h=[0x67452301,0xEFCDAB89,0x98BADCFE,0x10325476,0xC3D2E1F0]
    hprev=[0x67452301,0xEFCDAB89,0x98BADCFE,0x10325476,0xC3D2E1F0]
    k=[0x5A827999,0x6ED9EBA1,0x8F1BBCDC,0xCA62C1D6]
    func=[lambda b,c,d:d^(b&(c^d)),lambda b,c,d:b^c^d,lambda b,c,d:(b&c)|(b&d)|(c&d),lambda b,c,d:b^c^d]
    return h,hprev,k,func

In [4]:
def sha1(text):
    '''
    Generates SHA-1 hash for a text input
    
    Parameters:
    text (string): Input text string.
        
    Returns:
    result (string): Hexadecimal SHA-1 hash
    '''
    
    # Initialisation of values
    h,hprev,k,func=initialise_parameters()

    # Function to convert text to hexadecimal content of sha1
    inp=text2blocks(text)

    # Iterating 1 block at a time.
    for block in range(0,len(inp),128):
        # Dividing the message block of hexadecimal numbers into 16 words of 32 bits decimal numbers.
        word_arr=[]
        for byte_no in range(0,128,8):
            word_arr.append(int(inp[block+byte_no:block+byte_no+8],base=16))
        
        # Calculating the weights and updating the hash (h)
        weight=[]
        for round_no in range(80):
            if round_no<16:
                weight.append(word_arr[round_no])
            else:
                weight.append(circular(weight[round_no-16]^weight[round_no-14]^weight[round_no-8]^weight[round_no-3],1))
            h=[(h[4]+func[round_no//20](h[1],h[2],h[3])+circular(h[0],5)+weight[round_no]+k[round_no//20])%2**32,
               h[0],circular(h[1],30),h[2],h[3]]
        for j in range(5):
            h[j]=(h[j]+hprev[j])%2**32
        hprev=h

    # Representing the hash value in hexadecimal
    result=''
    for i in range(5):
        result+=hex(h[i])[2:].zfill(8)
    return result

In [5]:
sha1('Deepanjan Saha Deepanjan Saha Deepanjan Saha Deepanjan Saha Deepanjan Saha')

'f601b627ec1cb6c91f5c7b991e4ddb01cc26ce64'

In [12]:
from tkinter import filedialog
from tkinter import *
import os

class window(Frame):
    def __init__(self,master=None):
        Frame.__init__(self,master)
        self.master=master
        self.data_string=StringVar()
        self.viewtype=StringVar()
        self.selected=StringVar()
        self.content=StringVar()
        self.output_hash=StringVar()
        self.filename=''
        self.hash_value=''
        self.paraLabel={'master':self.master,'bg':'#000000','fg':'#ffffff','font':"Helvetica 13 bold italic"}
        
    def main_page(self):
        self.master.geometry("550x400")
        self.master.configure(background="black")
        
        label_text=Label(**self.paraLabel,text="Enter text here")
        label_text.place(x=50,y=50)
        
        input_text=Entry(self.master,textvariable=self.data_string)
        input_text.place(x=180,y=50)
        
        file_text=Label(**self.paraLabel,text="Browse for file")
        file_text.place(x=50,y=80)
        
        Browse_button=Button(self.master,text="Browse",command=self.get_file)
        Browse_button.place(x=180,y=80)
        
        selected_file=Label(**self.paraLabel,textvariable=self.selected)
        selected_file.place(x=50,y=110)
        
        hash_button=Button(self.master,text="Generate Hash",command=self.sha1_generator)
        hash_button.place(x=250,y=140)

        view_type=Label(**self.paraLabel,textvariable=self.viewtype)
        view_type.place(x=50,y=200)
        
        content=Label(**self.paraLabel,textvariable=self.content)
        content.place(x=50,y=240)
        
        hash_value=Label(**self.paraLabel,textvariable=self.output_hash)
        hash_value.place(x=50,y=280)
        
        clear=Button(self.master,text="Clear Result",command=self.clear)
        clear.place(x=50,y=340)
        
        hash_value=Button(self.master,text="copy Hash",command=self.copy)
        hash_value.place(x=150,y=340)
        
    def copy(self):
        print('inside copy')
        self.master.clipboard_clear()
        self.master.clipboard_append(self.hash_value)
        self.master.update()
        
    def clear(self):
        self.viewtype.set('')
        self.content.set('')
        self.output_hash.set('')

    def get_file(self):
        self.data_string.set('')
        self.filename =  filedialog.askopenfilename(initialdir = os.getcwd(),title = "Select file",
                                               filetypes = (("PDF files","*.pdf"),("jpeg files","*.jpg"),
                                                            ("all files","*.*")))
        file_name=self.filename.rpartition('/')[2]
        if len(file_name)>50:
            self.selected.set('Selected file: {}...'.format(file_name[:50]))
        else:
            self.selected.set('Selected file: {}'.format(file_name))
        
    def sha1_generator(self):
        if len(self.data_string.get())==0 and len(self.filename)==0:
            return
        elif len(self.filename)!=0:
            print(self.filename)
            data,self.filename=self.filename,''
            file=open(data,'rb')
            data=str(file.read())
            file.close()
            self.viewtype.set('TYPE:    FILE')
        else:
            data=self.data_string.get()
            self.data_string.set('')
            self.viewtype.set('TYPE:    TEXT')
        self.hash_value=sha1(data)
        if len(data)>50:
            self.content.set('CONTENT:    {}...'.format(data[:40]))
        else:
            self.content.set('CONTENT:    {}'.format(data))
        self.output_hash.set('HASH:    {}'.format(self.hash_value))
        
        
root = Tk()
app=window(root)
app.main_page()
root.mainloop()

inside copy
